importing libraies

In [0]:
import pandas as pd
import numpy as np 
import seaborn as sns
import pandas_profiling

connecting google drive to colab

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


saving dataset into dataframe

In [0]:
path='/content/drive/My Drive/Colab Notebooks/'
dataset = pd.read_csv(path+'clinvar_conflicting.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


removing coulmns with more than 25% missing values

In [0]:
dataset = dataset.drop(dataset.loc[:,list((100*(dataset.isnull().sum()/len(dataset.index))>=25))].columns, 1)

droping the remaing NAs

In [0]:
dataset = dataset.dropna()

removing unwanted coulmns 

In [0]:
dataset = dataset.drop(["CLNDISDB", "CLNHGVS", "MC", "Feature"], axis=1) 

EDA using pandas profiling

In [0]:
pandas_profiling.ProfileReport(dataset)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,28
Number of observations,44572
Total Missing (%),0.0%
Total size in memory,9.5 MiB
Average record size in memory,224.0 B
Numeric,8
Categorical,15
Boolean,2
Date,0
Text (Unique),0
Rejected,3


function for model evaluation

In [0]:
def report(y_actual, y_predicted):
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, auc, roc_curve
    print("Accuracy = ", accuracy_score(y_actual, y_pred))
    print("Precision = " ,precision_score(y_actual, y_pred))
    print("Recall = " ,recall_score(y_actual, y_pred))
    print("F1 Score = " ,f1_score(y_actual, y_pred))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_actual, y_pred)
    print("AUC Score =", auc(false_positive_rate, true_positive_rate))
    pass

X- indipendent variables y- dependent variable

In [0]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Under sampling using random undersampler

In [0]:
from imblearn.under_sampling import RandomUnderSampler
randomsample = RandomUnderSampler(random_state=2)
X, y = randomsample.fit_resample(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


converting X from np.arry into dataframe

In [0]:
X= pd.DataFrame(X) 

One hot encoding all the variables

In [0]:
X=pd.get_dummies(X, drop_first=True)  

In [0]:
X

,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,0_10,0_11,0_12,0_13,0_14,0_15,0_16,0_16,0_17,0_18,0_19,0_20,0_21,0_22,0_X,1_215028,1_218471,1_223666,1_223669,1_224608,1_225629,1_225662,1_226053,1_226090,1_226141,1_228343,1_228362,1_228401,1_228417,1_231038,1_231042,1_231050,...,25_15.676337,25_15.700142999999999,25_15.703435,25_15.705637,25_15.801829000000001,25_15.835087,25_15.873268,25_16.073826,25_16.101728,25_16.145831,25_16.335544,25_16.621753,25_16.707674,25_17.006708,25_17.232469000000002,25_17.756159,25_17.758349,25_17.772809,25_19.346589,25_20.253235999999998,25_20.677397,25_21.089312,25_21.704929,25_22.161135,25_22.200799,25_22.744774,25_23.295431,25_23.595398,25_23.817674,25_24.307762,25_24.363818,25_26.885931,25_27.805753999999997,25_27.988112,25_28.764711,25_29.845803999999998,25_30.852390000000003,25_32.543317,25_32.693999,25_33.935525
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21789,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21790,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21792,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Implimenting SVD

In [0]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=200)
X = svd.fit_transform(X)

Spiting into train test

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)

Adaboost classifier

In [0]:
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(n_estimators = 86, learning_rate = 1.0)
# Train Adaboost Classifer
adaboost = adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
report(y_test, y_pred)

Accuracy =  0.6405138793301216
Precision =  0.6316254416961131
Recall =  0.6611188164586222
F1 Score =  0.6460356900835781
AUC Score = 0.6406686978467974


Hyper parameter tuning

In [0]:
ada_params = {
        'n_estimators' : range(1, 100, 5),
        'learning_rate' : np.linspace(1e-16,1,3)
        }

In [0]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(adaboost, param_distributions=ada_params, n_iter=10, scoring='accuracy', n_jobs= -1, verbose=3, random_state=50, cv = 10 )
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  55 out of 100 | elapsed:   59.9s remaining:   49.0s
[Parallel(n_jobs=-1)]: Done  89 out of 100 | elapsed:  1.5min remaining:   11.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                base_estimator=None,
                                                learning_rate=1.0,
                                                n_estimators=50,
                                                random_state=None),
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'learning_rate': array([1.e-16, 5.e-01, 1.e+00]),
                                        'n_estimators': range(1, 100, 5)},
                   pre_dispatch='2*n_jobs', random_state=50, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=3)

In [0]:
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best hyperparameters:
{'n_estimators': 86, 'learning_rate': 1.0}


Gradient Boosting Classifier

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
gradient = GradientBoostingClassifier(n_estimators = 400, max_depth = 2, learning_rate = 1)
gradient = gradient.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gradient.predict(X_test)
report(y_test, y_pred)

Accuracy =  0.6313374627208075
Precision =  0.6256781193490054
Recall =  0.6398520573277855
F1 Score =  0.6326857142857142
AUC Score = 0.6314014384999583


hyper paramter tuning

In [0]:
gb_parameters = {
    "n_estimators":range(50, 500, 50),
    "max_depth":range(1, 9, 1),
    "learning_rate":range(1, 100, 10)
}

In [0]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(gradient, param_distributions=gm_parameters, n_iter=param_comb, scoring='accuracy', n_jobs= -1, verbose=3, random_state=50, cv = 10 )
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  55 out of 100 | elapsed:  5.1min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done  89 out of 100 | elapsed: 10.0min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 15.6min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                    

In [0]:
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best hyperparameters:
{'n_estimators': 400, 'max_depth': 2, 'learning_rate': 1}


XGboost classifier

In [0]:
from xgboost import XGBClassifier
xgboostoost = XGBClassifier(tree_method = 'gpu_hist', n_estimators = 91, min_child_weight = 8, max_depth = 1, learning_rate = 0.5, gamma = 3, eta = 0.16000000000000003)
xgb.fit(X_train, y_train)   
# Predicting the Test set results
y_pred = xgboost.predict(X_test)
report(y_test, y_pred)

Accuracy =  0.6549667354897912
Precision =  0.6487584650112866
Recall =  0.6643550624133149
F1 Score =  0.6564641388761991
AUC Score = 0.65503727619755


hyper paramter tuning

In [0]:
xgb_params = {
        'min_child_weight': range(1, 10, 1),
        'gamma': range(1, 5, 1),
        'max_depth': range(1, 5, 10),
        "n_estimators": range(1, 100, 10),
        'learning_rate': np.linspace(1e-16, 1, 3),
        'eta': np.arange(0.01, 0.2, 0.05)
        
        }
param_comb = 50

In [0]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(xgb, param_distributions=xgb_params, n_iter=param_comb, scoring='accuracy', n_jobs= -1, verbose=3, random_state=50, cv = 10 )
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.5min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=...
                                           verbosity=1),
                   iid='warn', n_iter=50, n_jobs=-1,
                   param_distributions={'eta': array([0.01, 0.06, 0.11, 0.16]),
                                

In [0]:
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best hyperparameters:
{'n_estimators': 91, 'min_child_weight': 8, 'max_depth': 1, 'learning_rate': 0.5, 'gamma': 3, 'eta': 0.16000000000000003}


lightgbm CLassifier

In [0]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.03
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 20
params['min_data'] = 5
params['max_depth'] = 20
lgbm = lgb.train(params, d_train, 500)
#Prediction
y_pred=lgbm.predict(X_test)

In [0]:
for i in range(0,4359):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [0]:
report(y_test, y_pred)

Accuracy =  0.6836430373938976
Precision =  0.6728395061728395
Recall =  0.7055016181229773
F1 Score =  0.6887835703001579
AUC Score = 0.6838072753638565


voting classifer

In [0]:
from sklearn.ensemble import VotingClassifier
majority = VotingClassifier(estimators=[('ada', ada),
                            ('xgb', xgb), ('gradient', gradient)],
                            voting='soft', weights=[1,1,2])

In [0]:
majority.fit(X_train, y_train)   
# Predicting the Test set results
y_pred = majority.predict(X_test)
report(y_test, y_pred)

Accuracy =  0.6393668272539573
Precision =  0.6332882273342354
Recall =  0.6490984743411928
F1 Score =  0.6410958904109589
AUC Score = 0.6394399475531101
